In [1]:
"""
Ipython notebook to test/document function that loads
the PRMS data file into a pandas dataframe
Author: John Volk, June 2016
Python 2.7.7
Pandas 0.18.1
"""
import pandas as pd
import os

In [2]:
def load_data_file(data_file):
    """ 
    Read the data file and load into a datetime indexed Pandas dataframe object
    INPUT: data_file = data file path (string)
    OUTPUT: df = Pandas dataframe of input time series data from data file with datetime index
    """
    ## valid input time series that can be put into a data file
    valid_input_variables = ('gate_ht', 
                             'humidity', 
                             'lake_elev', 
                             'pan_evap',
                             'precip', 
                             'rain_day', 
                             'runoff', 
                             'snowdepth', 
                             'solrad', 
                             'tmax', 
                             'tmin', 
                             'wind_speed')
    #### starting list of names for header in dataframe
    column_list = ['year', 
                   'month', 
                   'day', 
                   'hh', 
                   'mm', 
                   'sec']
    ## append to header list the variables present in the file
    with open(data_file, 'r') as inf:
        for idx,l in enumerate(inf):
            if idx == 0: ## first line always string identifier of the file- may use later
                data_head = l.rstrip()
            elif l.startswith('/'): ## comment lines
                continue
            if l.startswith(valid_input_variables): ## header lines with name and number of input variables
                h = l.split() ## split line into list, first element name and second number of columns
                if int(h[1]) > 1: ## more than one input time series of a particular variable
                    for el in range(int(h[1])):
                        tmp = '{var_name} {var_ind}'.format(var_name=h[0], var_ind=el+1)
                        column_list.append(tmp)
                elif int(h[1]) == 1:
                    column_list.append(h[0])
            if l.startswith('#'): ## end of header info and begin time series input data
                skip_line = idx+1
                break
    ## read data file into pandas dataframe object with correct header names
    missing_value= -999 ## missing data representation
    df = pd.read_csv(data_file, header=-1, skiprows=skip_line, 
                     delim_whitespace=True, na_values=[missing_value]) ## read file
    df.columns = column_list  ## apply correct header names using metadata retrieved from file
    date = pd.Series(pd.to_datetime(df.year*10000+df.month*100+df.day, format='%Y%m%d'), 
                     index=df.index) ## create date column
    df.index = pd.to_datetime(date) ## make the df index the datetime 
    df.drop(['year', 'month', 'day', 'hh', 'mm', 'sec'], axis=1, inplace=True) ## drop unneeded columns
    df.columns.name = 'input variables' ; df.index.name = 'date' ## name dataframe axes (index,columns)
    return df

In [3]:
data_file = os.path.join(os.getcwd(),'..','models/lbcd/lbcd_2.data') 

In [4]:
load_data_file(data_file).head()

input variables,runoff 1,runoff 2,runoff 3,precip,tmax,tmin
date,,,,,,
1992-10-01,1.2,1.9,NaN,0.0,77,52.0
1992-10-02,1.3,2.0,NaN,0.0,75,53.0
1992-10-03,1.4,2.1,NaN,0.0,62,44.0
1992-10-04,1.5,2.2,NaN,0.0,62,34.0
1992-10-05,1.5,2.2,NaN,0.0,62,35.0


In [6]:
import time
def time_it(f, *args):
    start = time.clock()
    f(*args)
    return (time.clock() - start)
n = 50
t = 0
for i in range(n):
    t += time_it(load_data_file, data_file)
print 'The average time it took to run the function over {n} runs was {avg} seconds'.format(n=n,avg=t/n)

The average time it took to run the function over 50 runs was 0.0036123 seconds
